---

# Очистка данных

***Мусор на входе модели $\to$ мусор на выходе.***

Ни одна модель машинного обучения не выдаст осмысленных результатов, если вы предоставите ей сырые данные. После формирования выборки данных их необходимо очистить.

In [68]:
#main
import pandas as pd
import numpy as np

#plot
from matplotlib import pyplot as plt
import seaborn as sns

#plot
from matplotlib import pyplot as plt
import seaborn as sns

#
from tqdm import tqdm 
import re

pd.set_option("display.max_columns", 300)


# Имеем :
PATH_TRAIN = './data/train_sber.csv'
PATH_MACRO = './data/macro.csv'

**У нас есть достаточно большой DataSet с кучей признаков много пропусков и мусорных значений. Будем обрабатывать!**

In [108]:
df = pd.read_csv(PATH_TRAIN)
df_makro = pd.read_csv(PATH_MACRO)
df_y = df['price_doc']
df.drop(columns='price_doc', axis=1, inplace=True)

**Для заполнения ошибочных значений надо вникнуть в суть признаков и определить их целесообразность  
В папке data/ файл data_dictionary.txt содержит описание всех признаков, посмотрим их**

В первую очередь можно заметить что первые 13 признаков это данные самой транзакции, а далее это различные характеристики района 

In [109]:
df.iloc[:, 0:13].head()

,id,timestamp,full_sq,life_sq,floor,max_floor,material,build_year,num_room,kitch_sq,state,product_type,sub_area
0,1,2011-08-20,43,27.0,4.0,NaN,NaN,NaN,NaN,NaN,NaN,Investment,Bibirevo
1,2,2011-08-23,34,19.0,3.0,NaN,NaN,NaN,NaN,NaN,NaN,Investment,Nagatinskij Zaton
2,3,2011-08-27,43,29.0,2.0,NaN,NaN,NaN,NaN,NaN,NaN,Investment,Tekstil'shhiki
3,4,2011-09-01,89,50.0,9.0,NaN,NaN,NaN,NaN,NaN,NaN,Investment,Mitino
4,5,2011-09-05,77,77.0,4.0,NaN,NaN,NaN,NaN,NaN,NaN,Investment,Basmannoe


Это важные признаки назавем их:

Признаки I очереди
---

id timestamp full_sq life_sq floor max_floor material build_year num_room kitch_sq state product_type sub_area

Для построения модели они болие важны

In [110]:
df_x1 = df.iloc[:, 0:13]

In [111]:
df[df['sub_area'] == 'Mitino'].iloc[:, 12:]

,sub_area,area_m,raion_popul,green_zone_part,indust_part,children_preschool,preschool_quota,preschool_education_centers_raion,children_school,school_quota,school_education_centers_raion,school_education_centers_top_20_raion,hospital_beds_raion,healthcare_centers_raion,university_top_20_raion,sport_objects_raion,additional_education_raion,culture_objects_top_25,culture_objects_top_25_raion,shopping_centers_raion,office_raion,thermal_power_plant_raion,incineration_raion,oil_chemistry_raion,radiation_raion,railroad_terminal_raion,big_market_raion,nuclear_reactor_raion,detention_facility_raion,full_all,male_f,female_f,young_all,young_male,young_female,work_all,work_male,work_female,ekder_all,ekder_male,ekder_female,0_6_all,0_6_male,0_6_female,7_14_all,7_14_male,7_14_female,0_17_all,0_17_male,0_17_female,16_29_all,16_29_male,16_29_female,0_13_all,0_13_male,0_13_female,raion_build_count_with_material_info,build_count_block,build_count_wood,build_count_frame,build_count_brick,build_count_monolith,build_count_panel,build_count_foam,build_count_slag,build_count_mix,raion_build_count_with_builddate_info,build_count_before_1920,build_count_1921-1945,build_count_1946-1970,build_count_1971-1995,build_count_after_1995,ID_metro,metro_min_avto,metro_km_avto,metro_min_walk,metro_km_walk,kindergarten_km,school_km,park_km,green_zone_km,industrial_km,water_treatment_km,cemetery_km,incineration_km,railroad_station_walk_km,railroad_station_walk_min,ID_railroad_station_walk,railroad_station_avto_km,railroad_station_avto_min,ID_railroad_station_avto,public_transport_station_km,public_transport_station_min_walk,water_km,water_1line,mkad_km,ttk_km,sadovoe_km,bulvar_ring_km,kremlin_km,big_road1_km,ID_big_road1,big_road1_1line,big_road2_km,ID_big_road2,railroad_km,railroad_1line,zd_vokzaly_avto_km,ID_railroad_terminal,bus_terminal_avto_km,ID_bus_terminal,oil_chemistry_km,nuclear_reactor_km,radiation_km,power_transmission_line_km,thermal_power_plant_km,ts_km,big_market_km,market_shop_km,fitness_km,swim_pool_km,ice_rink_km,stadium_km,basketball_km,hospice_morgue_km,detention_facility_km,public_healthcare_km,university_km,workplaces_km,shopping_centers_km,office_km,additional_education_km,preschool_km,big_church_km,church_synagogue_km,mosque_km,theater_km,museum_km,exhibition_km,catering_km,ecology,green_part_500,prom_part_500,office_count_500,office_sqm_500,trc_count_500,trc_sqm_500,cafe_count_500,cafe_sum_500_min_price_avg,cafe_sum_500_max_price_avg,cafe_avg_price_500,cafe_count_500_na_price,cafe_count_500_price_500,cafe_count_500_price_1000,cafe_count_500_price_1500,cafe_count_500_price_2500,cafe_count_500_price_4000,cafe_count_500_price_high,big_church_count_500,church_count_500,mosque_count_500,leisure_count_500,sport_count_500,market_count_500,green_part_1000,prom_part_1000,office_count_1000,office_sqm_1000,trc_count_1000,trc_sqm_1000,cafe_count_1000,cafe_sum_1000_min_price_avg,cafe_sum_1000_max_price_avg,cafe_avg_price_1000,cafe_count_1000_na_price,cafe_count_1000_price_500,cafe_count_1000_price_1000,cafe_count_1000_price_1500,cafe_count_1000_price_2500,cafe_count_1000_price_4000,cafe_count_1000_price_high,big_church_count_1000,church_count_1000,mosque_count_1000,leisure_count_1000,sport_count_1000,market_count_1000,green_part_1500,prom_part_1500,office_count_1500,office_sqm_1500,trc_count_1500,trc_sqm_1500,cafe_count_1500,cafe_sum_1500_min_price_avg,cafe_sum_1500_max_price_avg,cafe_avg_price_1500,cafe_count_1500_na_price,cafe_count_1500_price_500,cafe_count_1500_price_1000,cafe_count_1500_price_1500,cafe_count_1500_price_2500,cafe_count_1500_price_4000,cafe_count_1500_price_high,big_church_count_1500,church_count_1500,mosque_count_1500,leisure_count_1500,sport_count_1500,market_count_1500,green_part_2000,prom_part_2000,office_count_2000,office_sqm_2000,trc_count_2000,trc_sqm_2000,cafe_count_2000,cafe_sum_2000_min_price_avg,cafe_sum_2000_max_price_avg,cafe_avg_price_2000,cafe_count_2000_na_price,cafe_count_2000_price_500,cafe_count_2000_price_1000,ca

___

Можно заметить что от sub_area до ID_metro признаки отнасительно района и все индетичны: выделим это в отдельный dataset И поработаем с ним позже

In [112]:
df_sub_area = df.loc[:,'sub_area': 'build_count_after_1995']
df.drop(columns=df.loc[:,'area_m': 'build_count_after_1995'].columns, axis=1, inplace=True)

In [54]:
df.loc[:, 'ID_metro':].head()

,ID_metro,metro_min_avto,metro_km_avto,metro_min_walk,metro_km_walk,kindergarten_km,school_km,park_km,green_zone_km,industrial_km,water_treatment_km,cemetery_km,incineration_km,railroad_station_walk_km,railroad_station_walk_min,ID_railroad_station_walk,railroad_station_avto_km,railroad_station_avto_min,ID_railroad_station_avto,public_transport_station_km,public_transport_station_min_walk,water_km,water_1line,mkad_km,ttk_km,sadovoe_km,bulvar_ring_km,kremlin_km,big_road1_km,ID_big_road1,big_road1_1line,big_road2_km,ID_big_road2,railroad_km,railroad_1line,zd_vokzaly_avto_km,ID_railroad_terminal,bus_terminal_avto_km,ID_bus_terminal,oil_chemistry_km,nuclear_reactor_km,radiation_km,power_transmission_line_km,thermal_power_plant_km,ts_km,big_market_km,market_shop_km,fitness_km,swim_pool_km,ice_rink_km,stadium_km,basketball_km,hospice_morgue_km,detention_facility_km,public_healthcare_km,university_km,workplaces_km,shopping_centers_km,office_km,additional_education_km,preschool_km,big_church_km,church_synagogue_km,mosque_km,theater_km,museum_km,exhibition_km,catering_km,ecology,green_part_500,prom_part_500,office_count_500,office_sqm_500,trc_count_500,trc_sqm_500,cafe_count_500,cafe_sum_500_min_price_avg,cafe_sum_500_max_price_avg,cafe_avg_price_500,cafe_count_500_na_price,cafe_count_500_price_500,cafe_count_500_price_1000,cafe_count_500_price_1500,cafe_count_500_price_2500,cafe_count_500_price_4000,cafe_count_500_price_high,big_church_count_500,church_count_500,mosque_count_500,leisure_count_500,sport_count_500,market_count_500,green_part_1000,prom_part_1000,office_count_1000,office_sqm_1000,trc_count_1000,trc_sqm_1000,cafe_count_1000,cafe_sum_1000_min_price_avg,cafe_sum_1000_max_price_avg,cafe_avg_price_1000,cafe_count_1000_na_price,cafe_count_1000_price_500,cafe_count_1000_price_1000,cafe_count_1000_price_1500,cafe_count_1000_price_2500,cafe_count_1000_price_4000,cafe_count_1000_price_high,big_church_count_1000,church_count_1000,mosque_count_1000,leisure_count_1000,sport_count_1000,market_count_1000,green_part_1500,prom_part_1500,office_count_1500,office_sqm_1500,trc_count_1500,trc_sqm_1500,cafe_count_1500,cafe_sum_1500_min_price_avg,cafe_sum_1500_max_price_avg,cafe_avg_price_1500,cafe_count_1500_na_price,cafe_count_1500_price_500,cafe_count_1500_price_1000,cafe_count_1500_price_1500,cafe_count_1500_price_2500,cafe_count_1500_price_4000,cafe_count_1500_price_high,big_church_count_1500,church_count_1500,mosque_count_1500,leisure_count_1500,sport_count_1500,market_count_1500,green_part_2000,prom_part_2000,office_count_2000,office_sqm_2000,trc_count_2000,trc_sqm_2000,cafe_count_2000,cafe_sum_2000_min_price_avg,cafe_sum_2000_max_price_avg,cafe_avg_price_2000,cafe_count_2000_na_price,cafe_count_2000_price_500,cafe_count_2000_price_1000,cafe_count_2000_price_1500,cafe_count_2000_price_2500,cafe_count_2000_price_4000,cafe_count_2000_price_high,big_church_count_2000,church_count_2000,mosque_count_2000,leisure_count_2000,sport_count_2000,market_count_2000,green_part_3000,prom_part_3000,office_count_3000,office_sqm_3000,trc_count_3000,trc_sqm_3000,cafe_count_3000,cafe_sum_3000_min_price_avg,cafe_sum_3000_max_price_avg,cafe_avg_price_3000,cafe_count_3000_na_price,cafe_count_3000_price_500,cafe_count_3000_price_1000,cafe_count_3000_price_1500,cafe_count_3000_price_2500,cafe_count_3000_price_4000,cafe_count_3000_price_high,big_church_count_3000,church_count_3000,mosque_count_3000,leisure_count_3000,sport_count_3000,market_count_3000,green_part_5000,prom_part_5000,office_count_5000,office_sqm_5000,trc_count_5000,trc_sqm_5000,cafe_count_5000,cafe_sum_5000_min_price_avg,cafe_sum_5000_max_price_avg,cafe_avg_price_5000,cafe_count_5000_na_price,cafe_count_5000_price_500,cafe_count_5000_price_1000,cafe_count_5000_price_1500,cafe_count_5000_price_2500,cafe_count_5000_price_4000,cafe_count_5000_price_high,big_church_count_5000,church_count_5000,mosque_count_5000,leisure_count_5000,sport_count_5000,market_count_5000,price_doc
0,1,2.590241,1.131260,13.5751

ID_metro и далие отсечка для болие глубоких признаков выделим это в 

Признаки II очереди
---
---

In [113]:
df_x2 = df.loc[:, 'ID_metro':]

## Итог
### визуального осмотра признаков

Это разделение на несколько dataset - ов

df_y - искомая переменная  
df_x1 - основные признаки  
df_x2 - второстепенные глубокие признаки  
df_sub_area - повторяющиеся данные района  

---
обеденним все действия по разделению dataset в одну функцию ниже

In [167]:
def split_df(df):
    try:
        df_y = df['price_doc']
    except KeyError:
        df_y = None
    
    df_x1 = df.loc[:, :'sub_area']
    df_sub_area = df.loc[:,'sub_area': 'build_count_after_1995'].groupby(by=['sub_area']).mean()
    df_x2 = df.loc[:, 'ID_metro':]
    
    return df_y, df_x1, df_x2, df_sub_area

In [168]:
df = pd.read_csv(PATH_TRAIN)

df_y, df_x1, df_x2, df_sub_area = split_df(df)

In [169]:
df_sub_area

,area_m,raion_popul,green_zone_part,indust_part,children_preschool,preschool_quota,preschool_education_centers_raion,children_school,school_quota,school_education_centers_raion,school_education_centers_top_20_raion,hospital_beds_raion,healthcare_centers_raion,university_top_20_raion,sport_objects_raion,additional_education_raion,culture_objects_top_25_raion,shopping_centers_raion,office_raion,full_all,male_f,female_f,young_all,young_male,young_female,work_all,work_male,work_female,ekder_all,ekder_male,ekder_female,0_6_all,0_6_male,0_6_female,7_14_all,7_14_male,7_14_female,0_17_all,0_17_male,0_17_female,16_29_all,16_29_male,16_29_female,0_13_all,0_13_male,0_13_female,raion_build_count_with_material_info,build_count_block,build_count_wood,build_count_frame,build_count_brick,build_count_monolith,build_count_panel,build_count_foam,build_count_slag,build_count_mix,raion_build_count_with_builddate_info,build_count_before_1920,build_count_1921-1945,build_count_1946-1970,build_count_1971-1995,build_count_after_1995
sub_area,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
Ajeroport,4.662813e+06,73007.0,0.109947,5.027201e-02,4348.0,2058.0,3.0,4231.0,4975.0,6.0,1.0,1020.0,0.0,1.0,4.0,3.0,0.0,3.0,14.0,1100773.0,502659.0,598114.0,9151.0,4737.0,4414.0,44166.0,22748.0,21418.0,19690.0,6156.0,13534.0,4348.0,2225.0,2123.0,4231.0,2201.0,2030.0,10478.0,5479.0,4999.0,234407.0,115098.0,119309.0,8098.0,4177.0,3921.0,332.0,31.0,2.0,0.0,256.0,12.0,30.0,0.0,1.0,0.0,332.0,2.0,25.0,246.0,39.0,20.0
Akademicheskoe,5.704502e+06,106445.0,0.043127,1.513462e-01,5506.0,926.0,6.0,5889.0,9501.0,6.0,0.0,830.0,4.0,0.0,7.0,0.0,0.0,1.0,10.0,1362363.0,637906.0,724457.0,12074.0,6198.0,5876.0,68518.0,34132.0,34386.0,25853.0,8422.0,17431.0,5506.0,2861.0,2645.0,5889.0,2990.0,2899.0,14297.0,7209.0,7088.0,291222.0,143474.0,147748.0,10653.0,5506.0,5147.0,374.0,81.0,0.0,0.0,205.0,9.0,78.0,0.0,1.0,0.0,374.0,0.0,0.0,304.0,21.0,49.0
Alekseevskoe,5.234028e+06,78418.0,0.044074,2.001468e-01,3761.0,1037.0,3.0,3975.0,4658.0,3.0,0.0,770.0,2.0,0.0,9.0,2.0,0.0,0.0,8.0,1359171.0,635195.0,723976.0,8186.0,4313.0,3873.0,52919.0,30743.0,22176.0,17313.0,5092.0,12221.0,3761.0,1955.0,1806.0,3975.0,2111.0,1864.0,9514.0,4891.0,4623.0,283678.0,140881.0,142797.0,7263.0,3806.0,3457.0,250.0,72.0,0.0,0.0,160.0,0.0,18.0,0.0,0.0,0.0,248.0,0.0,17.0,186.0,40.0,5.0
Altuf'evskoe,3.294352e+06,55125.0,0.024333,2.982046e-01,3559.0,857.0,2.0,4048.0,3669.0,3.0,0.0,NaN,1.0,0.0,2.0,0.0,0.0,3.0,3.0,78418.0,40148.0,38270.0,8140.0,4241.0,3899.0,34611.0,17635.0,16976.0,12374.0,3675.0,8699.0,3559.0,1841.0,1718.0,4048.0,2128.0,1920.0,9115.0,4791.0,4324.0,20342.0,10622.0,9720.0,7148.0,3732.0,3416.0,119.0,24.0,0.0,0.0,47.0,1.0,47.0,0.0,0.0,0.0,119.0,0.0,0.0,72.0,42.0,5.0
Arbat,2.081628e+06,28179.0,0.009851,0.000000e+00,1379.0,1104.0,3.0,1423.0,5155.0,4.0,0.0,562.0,1.0,0.0,5.0,1.0,1.0,8.0,37.0,741887.0,336985.0,404902.0,2994.0,1569.0,1425.0,17209.0,9155.0,8054.0,7976.0,2798.0,5178.0,1379.0,721.0,658.0,1423.0,761.0,662.0,3360.0,1750.0,1610.0,150479.0,73186.0,77293.0,2646.0,1409.0,1237.0,251.0,3.0,0.0,0.0,228.0,12.0,7.0,0.0,0.0,1.0,251.0,156.0,26.0,30.0,15.0,24.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Vyhino-Zhulebino,1.509487e+07,219609.0,0.300284,3.349371e-02,10167.0,7610.0,10.0,10573.0,16049.0,11.0,0.0,1322.0,3.0,0.0,11.0,11.0,0.0,8.0,1.0,1318695.0,607126.0,711569.0,22341.0,11471.0,10870.0,140182.0,71360.0,68822.0,57086.0,19275.0,37811.0,10167.0,5279.0,4888.0,10573.0,5409.0,5164.0,25474.0,13030.0,12444.0,276143.0,137389.0,138754.0,19389.0,9977.0,9412.0,567.0,112.0,81.0,5.0,38.0,8.0,320.0,0.0,2.0,1.0,566.0,5.0,54.0,203.0,210.0,94.0
Zamoskvorech'e,4.280227e+06,55590.0,0.019885,1.800000e-09,3344.0,2224.0,4.0,3681.0,4487.0,4.0,0.0,1815.0,2.0,3.0,13.0,1.0,0.0,5.0,73.0,108171.0,49958.0,58213.0,7561.0,3682